In [1]:
from pathlib import Path
import bvhio.bvhio as bvhio
from bvhio.bvhio import BvhJoint, Joint, Pose
import plotly.express as px
import plotly.graph_objects as go
from typeguard import typechecked
from jaxtyping import Int, Float, Bool, Num, jaxtyped
from pydantic import BaseModel
from typing import Optional, Union
from plotly.graph_objects import layout
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.axes import Axes
import matplotlib.gridspec as gridspec
import numpy as np
from typing import cast
from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output
import ipywidgets as widgets
from IPython.display import clear_output

number = Union[int, float]
Color = tuple[int, int, int] | str
Step = layout.slider.Step
Slider = layout.Slider
NDArray = np.ndarray

In [2]:
b = bvhio.readAsBvh("172_jump_4.bvh")

In [3]:
print("Name\tIndex\tDepth")
for joint, index, depth in b.Root.layout():
    n = joint.Name
    print(f"{n}\t{index}\t{depth}")

Name	Index	Depth
Hips	0	0
Spine1	1	1
Spine2	2	2
Spine3	3	3
Spine4	4	4
Neck	5	5
Head	6	6
HeadTip	7	7
RightShoulder	8	5
RightArm	9	6
RightForeArm	10	7
RightHand	11	8
RightFinger1Metacarpal	12	9
RightFinger1Proximal	13	10
RightFinger1Distal	14	11
RightFinger1Tip	15	12
RightFinger5Metacarpal	16	9
RightFinger5Proximal	17	10
RightFinger5Medial	18	11
RightFinger5Distal	19	12
RightFinger5Tip	20	13
RightFinger4Metacarpal	21	9
RightFinger4Proximal	22	10
RightFinger4Medial	23	11
RightFinger4Distal	24	12
RightFinger4Tip	25	13
RightFinger3Metacarpal	26	9
RightFinger3Proximal	27	10
RightFinger3Medial	28	11
RightFinger3Distal	29	12
RightFinger3Tip	30	13
RightFinger2Metacarpal	31	9
RightFinger2Proximal	32	10
RightFinger2Medial	33	11
RightFinger2Distal	34	12
RightFinger2Tip	35	13
LeftShoulder	36	5
LeftArm	37	6
LeftForeArm	38	7
LeftHand	39	8
LeftFinger5Metacarpal	40	9
LeftFinger5Proximal	41	10
LeftFinger5Medial	42	11
LeftFinger5Distal	43	12
LeftFinger5Tip	44	13
LeftFinger4Metacarpal	45	9
LeftFinger4Prox

```txt
Hips
├── Spine1
│   ├── Spine2
│       ├── Spine3
│           ├── Spine4
│               ├── Neck
│               │   ├── Head
│               │       └── HeadTip
│               ├── RightShoulder
│               │   ├── RightArm
│               │       ├── RightForeArm
│               │           ├── RightHand
│               │               ├── RightFinger1Metacarpal
│               │               │   ├── RightFinger1Proximal
│               │               │       ├── RightFinger1Distal
│               │               │           └── RightFinger1Tip
│               │               ├── RightFinger5Metacarpal
│               │               │   ├── RightFinger5Proximal
│               │               │       ├── RightFinger5Medial
│               │               │           ├── RightFinger5Distal
│               │               │               └── RightFinger5Tip
│               │               ├── RightFinger4Metacarpal
│               │               │   ├── RightFinger4Proximal
│               │               │       ├── RightFinger4Medial
│               │               │           ├── RightFinger4Distal
│               │               │               └── RightFinger4Tip
│               │               ├── RightFinger3Metacarpal
│               │               │   ├── RightFinger3Proximal
│               │               │       ├── RightFinger3Medial
│               │               │           ├── RightFinger3Distal
│               │               │               └── RightFinger3Tip
│               │               ├── RightFinger2Metacarpal
│               │                   ├── RightFinger2Proximal
│               │                       ├── RightFinger2Medial
│               │                           ├── RightFinger2Distal
│               │                               └── RightFinger2Tip
│               ├── LeftShoulder
│                   ├── LeftArm
│                       ├── LeftForeArm
│                           ├── LeftHand
│                               ├── LeftFinger5Metacarpal
│                               │   ├── LeftFinger5Proximal
│                               │       ├── LeftFinger5Medial
│                               │           ├── LeftFinger5Distal
│                               │               └── LeftFinger5Tip
│                               ├── LeftFinger4Metacarpal
│                               │   ├── LeftFinger4Proximal
│                               │       ├── LeftFinger4Medial
│                               │           ├── LeftFinger4Distal
│                               │               └── LeftFinger4Tip
│                               ├── LeftFinger3Metacarpal
│                               │   ├── LeftFinger3Proximal
│                               │       ├── LeftFinger3Medial
│                               │           ├── LeftFinger3Distal
│                               │               └── LeftFinger3Tip
│                               ├── LeftFinger2Metacarpal
│                               │   ├── LeftFinger2Proximal
│                               │       ├── LeftFinger2Medial
│                               │           ├── LeftFinger2Distal
│                               │               └── LeftFinger2Tip
│                               ├── LeftFinger1Metacarpal
│                                   ├── LeftFinger1Proximal
│                                       ├── LeftFinger1Distal
│                                           └── LeftFinger1Tip
├── RightThigh
│   ├── RightShin
│       ├── RightFoot
│           ├── RightToe
│               └── RightToeTip
├── LeftThigh
    ├── LeftShin
        ├── LeftFoot
            ├── LeftToe
                └── LeftToeTip
```

In [4]:
MAX_DEPTH = 8
joints: list[BvhJoint] = []
for joint, index, depth in b.Root.layout():
    if depth <= MAX_DEPTH:
        joints.append(joint)
joints = list(filter(lambda j: "Toe" not in j.Name, joints))
for j in joints:
    print(j.Name)
print(len(joints))

Hips
Spine1
Spine2
Spine3
Spine4
Neck
Head
HeadTip
RightShoulder
RightArm
RightForeArm
RightHand
LeftShoulder
LeftArm
LeftForeArm
LeftHand
RightThigh
RightShin
RightFoot
LeftThigh
LeftShin
LeftFoot
22


In [5]:
class Joint(BaseModel):
    index: int
    opposite_index: Optional[int] = None
    name: str
    color: Color


class Bone(BaseModel):
    joint1: Joint
    joint2: Joint
    name: str
    color: Color

    @staticmethod
    def from_indexes(joints: list[Joint], idx_1: int, idx_2: int, name: str,
                     color: Color) -> "Bone":
        return Bone(joint1=joints[idx_1],
                    joint2=joints[idx_2],
                    name=name,
                    color=color)


# https://plotly.com/python-api-reference/generated/plotly.graph_objects.scatter3d.marker.html
# plotly.graph_objects.scatter3d.Marker
def to_rgb_str(color: tuple[int, int, int]) -> str:
    return f"rgb({color[0]},{color[1]},{color[2]})"

- Hips
- Spine1
- Spine2
- Spine3
- Spine4
- Neck
- Head
- HeadTip
- RightShoulder
- RightArm
- RightForeArm
- RightHand
- LeftShoulder
- LeftArm
- LeftForeArm
- LeftHand
- RightThigh
- RightShin
- RightFoot
- LeftThigh
- LeftShin
- LeftFoot

In [6]:
def print_stuff(joint:BvhJoint):
    print(joint.Channels)
    kf = joint.Keyframes[201]
    print(joint.Offset)
    print(kf.Position) # if there's no position channel, use offset as position (weird)
    print(kf.Rotation)

print_stuff(joints[0])
print_stuff(joints[1])
print_stuff(joints[2])
print_stuff(joints[10])

['Xposition', 'Yposition', 'Zposition', 'Yrotation', 'Xrotation', 'Zrotation']
vec3(            0,      91.3446,            0 )
vec3(       2.1407,      -7.5077,      -18.061 )
quat(     0.960355,     0.262053,   -0.0615638,   -0.0725047 )
['Yrotation', 'Xrotation', 'Zrotation']
vec3(            0,       8.4838,      -3.9483 )
vec3(            0,       8.4838,      -3.9483 )
quat(    0.0210957,    0.0116462,     0.132615,    -0.990875 )
['Yrotation', 'Xrotation', 'Zrotation']
vec3(            0,        -7.55,            0 )
vec3(            0,        -7.55,            0 )
quat(     0.990526,    -0.135112,    0.0118659,    0.0214931 )
['Yrotation', 'Xrotation', 'Zrotation']
vec3(            0,     -27.1022,            0 )
vec3(            0,     -27.1022,            0 )
quat(     0.599901,    -0.515715,    0.0204102,    -0.611343 )


In [7]:
COLOR_SPINE = to_rgb_str((138, 201, 38))  # green, spine & head
COLOR_ARMS = to_rgb_str((255, 202, 58))  # yellow, arms & shoulders
COLOR_LEGS = to_rgb_str((25, 130, 196))  # blue, legs & hips
CIRCLE_SIZE = 4
LINE_WIDTH = 3
s_bvh_joints = [
    Joint(index=0, name="Hips", color=COLOR_SPINE),
    Joint(index=1, name="Spine1", color=COLOR_SPINE),
    Joint(index=2, name="Spine2", color=COLOR_SPINE),
    Joint(index=3, name="Spine3", color=COLOR_SPINE),
    Joint(index=4, name="Spine4", color=COLOR_SPINE),
    Joint(index=5, name="Neck", color=COLOR_SPINE),
    Joint(index=6, name="Head", color=COLOR_SPINE),
    Joint(index=7, name="HeadTip", color=COLOR_SPINE),
    Joint(index=8, name="RightShoulder", color=COLOR_ARMS),
    Joint(index=9, name="RightArm", color=COLOR_ARMS),
    Joint(index=10, name="RightForeArm", color=COLOR_ARMS),
    Joint(index=11, name="RightHand", color=COLOR_ARMS),
    Joint(index=12, name="LeftShoulder", color=COLOR_ARMS),
    Joint(index=13, name="LeftArm", color=COLOR_ARMS),
    Joint(index=14, name="LeftForeArm", color=COLOR_ARMS),
    Joint(index=15, name="LeftHand", color=COLOR_ARMS),
    Joint(index=16, name="RightThigh", color=COLOR_LEGS),
    Joint(index=17, name="RightShin", color=COLOR_LEGS),
    Joint(index=18, name="RightFoot", color=COLOR_LEGS),
    Joint(index=19, name="LeftThigh", color=COLOR_LEGS),
    Joint(index=20, name="LeftShin", color=COLOR_LEGS),
    Joint(index=21, name="LeftFoot", color=COLOR_LEGS),
]

In [8]:
@jaxtyped(typechecker=typechecked)
def joint_to_stacked(joint: BvhJoint) -> Num[NDArray, "N 3"]:
    """
    Convert a joint to a numpy matrix of shape (N, 3), where N is the number of keyframes.
    """
    return np.stack([np.array(joint.Keyframes[i].Position) for i in range(len(joint.Keyframes))])

@jaxtyped(typechecker=typechecked)
def stack_many(joints: list[BvhJoint]) -> Num[NDArray, "N J 3"]:
    """
    Stack the keyframes of many joints into a single numpy matrix, where N is the number of keyframes and J is the number of joints.

    Note that the order of the joints is preserved in the second dimension of the output.
    Every joint in the list should have the same number of keyframes.
    """
    return np.stack([joint_to_stacked(j) for j in joints], axis=1)

kps = stack_many(joints)
display(kps.shape)

(1051, 22, 3)

In [9]:
fw = go.FigureWidget()


def plot_frame(kps: Num[NDArray, "N J 3"], index: int):
    global fw
    global is_first
    assert 0 <= index < kps.shape[0]
    fig = go.Figure()
    sel = kps[index]
    # reverse the upright axis
    sel[:, 2] = -sel[:, 2]

    scatters = [
        go.Scatter3d(x=[sel[j.index, 0]],
                     y=[sel[j.index, 1]],
                     z=[sel[j.index, 2]],
                     mode='markers',
                     marker=dict(size=CIRCLE_SIZE, color=j.color),
                     name=j.name) for j in s_bvh_joints
    ]
    fig.add_traces(scatters)
    fw = go.FigureWidget(fig)
    # if there's a JavaScript error
    # restart Visual Studio Code (or use `.show()` method?)
    fw.update_layout(height=600)
    display(fw)
    return fig


slider = widgets.IntSlider(min=0,
                           max=kps.shape[0] - 1,
                           step=1,
                           value=0,
                           continue_update=False)

p = interactive(plot_frame, kps=fixed(kps), index=slider)
display(p)

interactive(children=(IntSlider(value=0, description='index', max=1050), Output()), _dom_classes=('widget-inte…